<a href="https://colab.research.google.com/github/yuliaplaksina/Machine-learning/blob/master/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 лабораторная

## Ансамбли моделей машинного обучения.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
import io
from google.colab import files
data = files.upload()

Saving heart.csv to heart (2).csv


In [3]:
data = pd.read_csv('heart.csv', sep=",")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Проверим, есть ли в наборе данных путсые значения

In [4]:
data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [5]:
data.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

Вынесем из общей таблицы целевой признак

In [6]:
X = data.loc[:, data.columns != 'target']
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [7]:
Y = data['target']
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

1. Разделим выборку на тренировочную и тестовую 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, random_state=1)

In [9]:
#Размер тренировочной выборки
X_train.shape, y_train.shape

((227, 13), (227,))

In [10]:
#Размер тестовой выборки
X_test.shape, y_test.shape

((76, 13), (76,))

In [0]:
bagging_tree = BaggingClassifier(DecisionTreeClassifier(random_state=1), n_estimators=100)

In [13]:
bagging_tree.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

In [14]:
bagging_tree_y_test = bagging_tree.predict(X_test)
bagging_tree_y_test

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [15]:
classification_report(y_test, bagging_tree_y_test, output_dict=True)["0"], \
classification_report(y_test, bagging_tree_y_test, output_dict=True)["1"]

({'f1-score': 0.6857142857142857,
  'precision': 0.6857142857142857,
  'recall': 0.6857142857142857,
  'support': 35},
 {'f1-score': 0.7317073170731707,
  'precision': 0.7317073170731707,
  'recall': 0.7317073170731707,
  'support': 41})

In [16]:
bagging_tree_n_range = np.array(range(10, 200, 10))
bagging_tree_tuned_parameters = [{'n_estimators': bagging_tree_n_range}]
bagging_tree_tuned_parameters

[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
         140, 150, 160, 170, 180, 190])}]

In [0]:
scoring = {
    'recall': make_scorer(recall_score), 
    'f1': make_scorer(f1_score), 
    'accuracy': make_scorer(accuracy_score)
}

In [24]:
bagging_tree_gs = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(random_state=1)), bagging_tree_tuned_parameters, cv=5, scoring=scoring, refit='f1')
bagging_tree_gs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                               min_samples_split=2,
                                                   

In [25]:
best_bagging = bagging_tree_gs.best_estimator_
best_bagging

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

In [26]:
bagging_tree_gs.best_score_

0.8772973024456084

In [27]:
bagging_tree_gs.best_params_

{'n_estimators': 110}

In [28]:
best_bagging.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=1,
                                                        splitter='best'),
       

In [29]:
best_bagging_y_test = best_bagging.predict(X_test)
best_bagging_y_test

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [30]:
classification_report(y_test, best_bagging_y_test, output_dict=True)["0"], \
classification_report(y_test, best_bagging_y_test, output_dict=True)["1"]

({'f1-score': 0.676470588235294,
  'precision': 0.696969696969697,
  'recall': 0.6571428571428571,
  'support': 35},
 {'f1-score': 0.7380952380952381,
  'precision': 0.7209302325581395,
  'recall': 0.7560975609756098,
  'support': 41})

In [0]:
extra_trees = ExtraTreesClassifier(random_state=1, n_estimators=100)

In [32]:
extra_trees.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [33]:
extra_trees_y_test = extra_trees.predict(X_test)
extra_trees_y_test

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [34]:
classification_report(y_test, extra_trees_y_test, output_dict=True)["0"], \
classification_report(y_test, extra_trees_y_test, output_dict=True)["1"]

({'f1-score': 0.6956521739130436,
  'precision': 0.7058823529411765,
  'recall': 0.6857142857142857,
  'support': 35},
 {'f1-score': 0.746987951807229,
  'precision': 0.7380952380952381,
  'recall': 0.7560975609756098,
  'support': 41})

In [35]:
extra_tree_n_range = np.array(range(10, 200, 10))
extra_tree_tuned_parameters = [{'n_estimators': extra_tree_n_range}]
extra_tree_tuned_parameters

[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
         140, 150, 160, 170, 180, 190])}]

In [36]:
extra_tree_gs = GridSearchCV(ExtraTreesClassifier(random_state=1), extra_tree_tuned_parameters, cv=5, scoring=scoring, refit='f1')
extra_tree_gs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=None,
                                            max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators='warn', n_jobs=None,
                                            oob_score=Fal...dom_state=1,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'n_estimators': array([ 10, 

In [37]:
best_extra_tree = extra_tree_gs.best_estimator_
best_extra_tree

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=140,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [38]:
extra_tree_gs.best_params_

{'n_estimators': 140}

In [39]:
extra_tree_gs.best_score_

0.8871624773257318

In [40]:
best_extra_tree.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=140,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [41]:
best_extra_tree_y_test = best_extra_tree.predict(X_test)
best_extra_tree_y_test

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [42]:
classification_report(y_test, best_extra_tree_y_test, output_dict=True)["0"], \
classification_report(y_test, best_extra_tree_y_test, output_dict=True)["1"]

({'f1-score': 0.6857142857142857,
  'precision': 0.6857142857142857,
  'recall': 0.6857142857142857,
  'support': 35},
 {'f1-score': 0.7317073170731707,
  'precision': 0.7317073170731707,
  'recall': 0.7317073170731707,
  'support': 41})

В двух сравниваемых методах лучше себя показал метод бэггинга